In [16]:
import src.simulation as sim
import numpy as np
import pandas as pd
import src.sample as smp

In [4]:
sample = smp.genSample('Basic Linear',250,tau=0.3,alpha=-0.15,beta=1,L=200,outlier=False,outlierMethod='Simple Outside', nOutliers=5,printPlot=False)

In [19]:
a = np.array([[0,10,2],[0,1,2]])

In [20]:
print(a)

[[ 0 10  2]
 [ 0  1  2]]


In [21]:
A = pd.DataFrame(columns=['e','d','f'],data=a)

In [23]:
A.mean()

e    0.0
d    5.5
f    2.0
dtype: float64

In [6]:
sample.mean()

Y            0.006322
X           -0.028706
Treatment    0.520000
Outlier      0.000000
dtype: float64